### Para esse script rodar a seguinte estrutura precisa executar antes:
####### Notepad: Carga Dados Compras 

### Mostrando o histórico das transações no Delta Lake - tabela Compras

In [0]:
%sql
DESCRIBE HISTORY '/user/hive/warehouse/compras'

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2023-07-25T18:39:04.000+0000,3749819615057738,andersonfaro@gmail.com,DELETE,"Map(predicate -> [""(cast(id#3470 as int) = 1)""])",null,List(851213765213230),0725-180139-s9tz1nlq,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 4851, numDeletedRows -> 1, scanTimeMs -> 3352, numAddedFiles -> 0, rewriteTimeMs -> 1481)",null,Databricks-Runtime/10.4.x-scala2.12
2,2023-07-25T18:38:52.000+0000,3749819615057738,andersonfaro@gmail.com,UPDATE,"Map(predicate -> [""(cast(id#2785 as int) = 4)""])",null,List(851213765213230),0725-180139-s9tz1nlq,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 4383, scanTimeMs -> 260, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 4111)",null,Databricks-Runtime/10.4.x-scala2.12
1,2023-07-25T18:38:37.000+0000,3749819615057738,andersonfaro@gmail.com,MERGE,"Map(predicate -> [""(cast(id#1927 as bigint) = id#1365L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(851213765213230),0725-180139-s9tz1nlq,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 7, executionTimeMs -> 7505, numTargetRowsInserted -> 7, scanTimeMs -> 2311, numTargetRowsUpdated -> 0, numOutputRows -> 7, numTargetChangeFilesAdded -> 0, numSourceRows -> 7, numTargetFilesRemoved -> 0, rewriteTimeMs -> 4956)",null,Databricks-Runtime/10.4.x-scala2.12
0,2023-07-25T18:38:20.000+0000,3749819615057738,andersonfaro@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [""date_order""], properties -> {})",null,List(851213765213230),0725-180139-s9tz1nlq,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


### Mostrando os dados de criação da tabela compras

In [0]:
%sql
DESCRIBE DETAIL '/user/hive/warehouse/compras'

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,6a94dac5-d86d-4fe1-9b91-c8bd49f51897,null,null,dbfs:/user/hive/warehouse/compras,2023-07-25T18:38:18.351+0000,2023-07-25T18:39:04.000+0000,List(date_order),6,9398,Map(),1,2


## Retornando(exibindo) a versão dos dados a posição do merge dos dados

In [0]:
%sql
SELECT * FROM delta. `/user/hive/warehouse/compras` VERSION AS OF 2

id,date_order,customer,product,unit,price
2,2021-03-22,Ana,AR-CONDICIONADO,6,121.6
4,2021-04-23,Sandra,Geladeira,8,313.0
5,2021-05-23,Tereza,FRIGOBAR,3,412.0
7,2021-07-25,Sofia,CADEIRA,1,342.3
3,2021-04-21,Sofia,FREEZER,7,415.4
6,2021-06-25,Carla,MESA,1,124.0
1,2021-01-23,Carlos,TV,5,238.0


## Mostrando os dados após o retorno das versões

In [0]:
%scala
spark.sql("select * from compras").show();

+---+----------+--------+---------------+----+-----+
 id|date_order|customer| product|unit|price|
+---+----------+--------+---------------+----+-----+
 2|2021-03-22| Ana|AR-CONDICIONADO| 6|121.6|
 4|2021-04-23| Sandra| Geladeira| 8|313.0|
 5|2021-05-23| Tereza| FRIGOBAR| 3|412.0|
 7|2021-07-25| Sofia| CADEIRA| 1|342.3|
 3|2021-04-21| Sofia| FREEZER| 7|415.4|
 6|2021-06-25| Carla| MESA| 1|124.0|
+---+----------+--------+---------------+----+-----+

## Clonando os dados gerados

In [0]:
%sql
CREATE TABLE delta. `/temporario/hive/clonado` CLONE delta. `/user/hive/warehouse/compras`

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2540306768249927> in <module>
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-2540306768249927> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("Q1JFQVRFIFRBQkxFIGRlbHRhLiBgL3RlbXBvcmFyaW8vaGl2ZS9jbG9uYWRvYCBDTE9ORSBkZWx0YS4gYC91c2VyL2hpdmUvd2FyZWhvdXNlL2NvbXByYXNg").decode())
      5     display(df)
      6     return df

/databricks/spark/python/pyspark/sql/session.py in sql(self, sqlQuery)
    775         [Row(f1=1, f2='row1'), Row(f1=2, f2='row2'), Row(f1=3, f2='row3')]
    776         """
--> 777         return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
    778 
    779     def table(self,

## Restaurando em definitivo os dados na tabela compras

In [0]:
%sql
RESTORE TABLE compras TO VERSION AS OF 1

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
10876,7,1,1,1578,1528


## Mostrando os dados restaurados

In [0]:
%scala
spark.sql("select * from compras").show();

+---+----------+--------+---------------+----+-----+
 id|date_order|customer| product|unit|price|
+---+----------+--------+---------------+----+-----+
 2|2021-03-22| Ana|AR-CONDICIONADO| 6|121.6|
 4|2021-04-23| Sandra| Geladeira| 8|313.0|
 5|2021-05-23| Tereza| FRIGOBAR| 3|412.0|
 7|2021-07-25| Sofia| CADEIRA| 1|342.3|
 3|2021-04-21| Sofia| FREEZER| 7|415.4|
 6|2021-06-25| Carla| MESA| 1|124.0|
 1|2021-01-23| Carlos| TV| 5|238.0|
+---+----------+--------+---------------+----+-----+